In [12]:
import os
import logging
import requests
from typing import Any, Dict, List, Union

from crewai import BaseLLM, Agent, Task, Crew
from crewai.tools import tool
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()

# 日志
logging.basicConfig(level=logging.INFO)

# 自定义 LLM 实现（完全绕开 CrewAI 内置 provider 机制）
class CustomLLM(BaseLLM):
    def __init__(self, model: str, base_url: str, api_key: str, temperature: float = 0.7):
        super().__init__(model=model, temperature=temperature)
        self.base_url = base_url.rstrip("/")
        self.api_key = api_key

    def call(
        self,
        messages: Union[str, List[Dict[str, str]]],
        *args: Any,
        **kwargs: Any
    ) -> str:
        # 如果是字符串，则转换成 openai 兼容的 messages
        if isinstance(messages, str):
            messages = [{"role": "user", "content": messages}]

        payload = {
            "model": self.model,
            "messages": messages
        }
        
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

        # 直接用 requests 发送 HTTP 请求，不走 litellm/OpenAI SDK
        response = requests.post(f"{self.base_url}/chat/completions", json=payload, headers=headers, timeout=30)
        response.raise_for_status()
        data = response.json()

        # 按照 OpenAI 兼容的返回结构返回
        return data["choices"][0]["message"]["content"]

    def supports_function_calling(self) -> bool:
        # 如不需要内置函数调用支持则返回 False
        return False

    def supports_stop_words(self) -> bool:
        return True

    def get_context_window_size(self) -> int:
        # 根据 model 支持调整
        return 8192

# 简单工具
@tool("Stock Price Lookup Tool")
def get_stock_price(ticker: str) -> float:
    """
    Fetches the latest simulated stock price for a given stock ticker symbol.
    Returns the price as a float. Raises a ValueError if the ticker is not found.
    """
    logging.info(f"Tool Call: get_stock_price for ticker '{ticker}'")
    store = {"AAPL": 178.15, "GOOGL": 1750.30, "MSFT": 425.50}
    price = store.get(ticker.upper())
    if price is None:
        raise ValueError(f"Ticker {ticker} not found.")
    return price

# 从环境变量读取
base_url = os.getenv("OPENAI_API_BASE")
api_key = os.getenv("OPENAI_API_KEY")

# 创建自定义 LLM
llm = CustomLLM(
    model="gemini-2.5-flash",  # 这里填你希望调用的模型名
    base_url=base_url,
    api_key=api_key,
)

# 创建 Agent
financial_analyst_agent = Agent(
    role="Senior Financial Analyst",
    goal="Report stock price using provided tools",
    backstory="You are an expert who answers succinctly.",
    llm=llm,
    tools=[get_stock_price],
    verbose=True,
    allow_delegation=False,
)

# 创建 Task
task = Task(
    description="What is the current simulated stock price for AAPL?",
    expected_output="A sentence stating the simulated stock price.",
    agent=financial_analyst_agent
)

# 运行 Crew
crew = Crew(agents=[financial_analyst_agent], tasks=[task], verbose=True)

def main():
    print("🚀 Running the Crew...")
    result = crew.kickoff()
    print("✅ Final result:\n", result)

if __name__ == "__main__":
    main()

🚀 Running the Crew...


╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  ba3248fd-9753-47a6-9437-0711199e7a9d                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: What is the current simulated stock price for AAPL?                                                      │
│  ID: d2a35b81-abc9-4be6-a6fd-2af90eee9571                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Financial Analyst                                                                                │
│                                                                                                                 │
│  Task: What is the current simulated stock price for AAPL?                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2026-01-18 22:20:01,235 - INFO - Tool Call: get_stock_price for ticker 'AAPL'


...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Financial Analyst                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The current simulated stock price for AAPL is 178.15.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│  What is the current simulated stock price for AAPL?                                                            │
│  Agent:                                                                                                         │
│  Senior Financial Analyst                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✅ Final result:
 The current simulated stock price for AAPL is 178.15.


╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  ba3248fd-9753-47a6-9437-0711199e7a9d                                                                           │
│  Final Output: The current simulated stock price for AAPL is 178.15.                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯